# Decision Tree Code

This code enables you to build a decision tree

In [23]:
import pandas as pd
import numpy as np
import os

# Code to Construct DataFrame

In [24]:
x_col = 'Floor Slipperiness'
y_col = 'Shoe Slipperiness'
value_map = {0:'No Fall',1:'Fall'}

# Test data Frame
# Altering this is reflected in the dataframe and the graphs produced, e,g, a 5x5 makes a 25 point graph
# a "1" means it's labelled as a "Fall" and "0" means it's labelled as a "No Fall"
# It assumes datapoints are labelled 0,1,2,3,4 etc. Feel free to add functionality to assign different values!
test = [
[1,1,1,1,1],
[1,1,1,1,1],
[1,1,0,1,1],
[0,0,0,1,1],
[0,0,1,1,1]]

# Make the data grid
df = pd.DataFrame(test)

# reset index so the above grip relates to the graph
df = df.reindex(index=df.index[::-1]).reset_index(drop=True)

df.reset_index(inplace=True)
# rename
df.rename({'index':y_col},inplace=True,axis='columns')

# Melt it (this turns the grid into a more usable dataframe)
# Have to extract the colums for the x_values while ignoring the column for the Shoe Slipperiness
col_list = list(df.columns)
# remove the y column name from the list (so we get just the x axis values)
col_list.remove(y_col)

df_format = pd.melt(df.reset_index(),id_vars=[y_col],value_vars=col_list)
df_format.rename({'variable':x_col},inplace=True,axis='columns')
# remap the value column
df_format['value']=df_format['value'].map(value_map)

# save it, Uncomment if you want this file (e.g. to put into an Excel file)
# df_format.to_csv('Slip_Data.csv',index=False)

# Display Output
df_format.head(5)

,Shoe Slipperiness,Floor Slipperiness,value
0,0,0,No Fall
1,1,0,No Fall
2,2,0,Fall
3,3,0,Fall
4,4,0,Fall


# Previous Code for Selecting Splits

In [91]:

def uniquecounts(df,col = 'value'):
    """
    Takes a dataframe and a column and counts the number of time each distinct entry apppears
    in that column before returning it
    """
    temp = df[col].value_counts().reset_index()
    temp.columns = [col,'Counts']
    return temp

def giniimpurity(df,col='value'):
    """
    Calculates the gini impurity score by calculating the probability of selecting an item from a group
    and multiplies it by the probability of it being assigned ot one of the other groups
    This is then repeated and summed to get a total purity
    """

    
    total=len(df)
    counts = uniquecounts(df,col=col)
    entries = counts[col].unique() # Number of unique entries
    imp=0.0
    for k1 in entries:
        p1 = counts.loc[counts[col]==k1,'Counts'].sum()/total
        for k2 in entries:
            if k1==k2:continue
            p2=counts.loc[counts[col]==k2,'Counts'].sum()/total
            imp+=p1*p2
    return imp

def dividedf(df,column,value):
    """
    This function splits a dataframe based on a column and a value
    If the value is a number it assigns all items greater or equal to it to one group
    and the rest to the second
    If the value is not a number then it assigns all items that exactly match that value to that group
    and the rest goes in the second group
    """
    # We work out if the value is a number or not
    val_type = df[column].tolist()[-1]
    if pd.api.types.is_number(val_type):
        mask = df[column]>=value
    else:
        mask = df[column]==value

    # Split into separate sets
    first_set = df.loc[mask,:].reset_index(drop=True)
    second_set = df.loc[~mask,:].reset_index(drop=True)
    return first_set, second_set

def findbest(df,scorer=giniimpurity,val_col='value'):
    """
    This function takes a dataframe and a way of scoring a split and then iterates over the dataframe
    The output is the best split groups alongside the column and values used to make that split
    """
    
    col_list = list(df.columns)
    # remove the value column
    col_list.remove(val_col)
    # What's the current score
    current_score = scorer(df,col=val_col)
    print("Initial Score: {}".format(current_score))

    
    # Iterate over the columns
    best_gain = 0.0
    best_criteria=None
    best_column=None
    for col in col_list:
        local_gain = 0.0
        local_column = None
        local_criteria = None
        unique_vals = df.loc[:,col].unique()
        for value in unique_vals:
            # Perform a split at that value
            set1, set2 = dividedf(df,col,value)
            # We weight the change by the fraction of each resulting set (i.e. smaller set has lower contribution)
            set1_fraction = len(set1)/len(df)

            # calculate the gain of impurity
            gain = current_score-set1_fraction*scorer(set1,col=val_col)-(1-set1_fraction)*scorer(set2,col=val_col)

            # Check if there's a gain, also we exclude data where there is no split (i.e. all data in one set)
            if gain>local_gain and len(set1)>0 and len(set2)>0:
                local_gain=gain
                local_column = col
                local_criteria = value
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain = gain
                best_column = col
                best_criteria = value
                # update criteria to be inbetween
        # To make the graph easier we assign the split as between the two values and not at it for the local best split
        # We will change this back for future builds where we are looking at classes and not continuous numbers
#         next_best = df.loc[df[local_column]<local_criteria,local_column].max()
#         local_criteria = (local_criteria+next_best)/2
        # Print local best set
#         print("For Column {}, best gain was {} for split at {}".format(local_column,round(local_gain,4),local_criteria))
    # Update global best to be between groups for better visualisation on the graphs
    next_best = df.loc[df[best_column]<best_criteria,best_column].max()
    best_criteria = (best_criteria+next_best)/2    
    # Print Overal Best Find
    print("\nBest selection was:")
    print("For Column {}, best gain was {} for split at {}".format(best_column,round(best_gain,4),best_criteria))
    
    # do best cut and return
    set1, set2 = dividedf(df,best_column,best_criteria)
    set1_fraction = len(set1)/len(df)
    set1_score = scorer(set1,col=val_col)
    set2_score = scorer(set2,col=val_col)
    weighted_score = set1_fraction*set1_score-(1-set1_fraction)*set2_score
    print("First Group ({} rows) Score: {} and Second Group ({} rows) Score: {}".format(len(set1),round(set1_score,4),
                                                                                        len(set2),round(set2_score,4)))
    
    return set1, set2, best_column, best_criteria

# Decision Tree Building Code

In [92]:
temp = uniquecounts(df_format).iloc[0,:]
temp

value     Fall
Counts      19
Name: 0, dtype: object

In [145]:
# We will want a class
# Should have __init__ function to store the data
# Should have __str__ function to print it out

class TreeNode:
    
    def __init__(self,depth,column=None,value=None,results=None,true_branch=None,false_branch=None):
        self.split_column = column
        self.split_value = value
        self.results = results
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.depth = depth
    
    def __str__(self):
        # Check if it's a result node
        if self.results==None:
            return str(self.depth*"    "+"{}. {}>={}?:\n".format(self.depth+1,self.split_column,self.split_value)+
                       str(self.true_branch) + "\n"+self.depth*"    "+"{}. {}<{}?:\n".format(self.depth+1,self.split_column,self.split_value) + str(self.false_branch))
        else: # If it is an edge/leaf, check if it is the true of false branch
            return str(self.depth*"    "+"Results -> {} : {} samples".format(self.results[0],self.results[1]))

    def predict(self,df):
        if self.results==None:
            if (df[self.split_column]>=self.split_value)==True:
                return self.true_branch.predict(df)
            else:
                return self.false_branch.predict(df)
        else:
            return self.results[0]
        
    def return_depth(self):
        if self.results==None:
            return max(self.true_branch.return_depth(),self.false_branch.return_depth())
        else:
            return self.depth
    
def buildTree(df,decision_function=giniimpurity,val_col='value',depth=0):
    """
    This function recursively builds a tree and returns it
    The returned built tree can be used to make predictions
    """
    score = decision_function(df,val_col)
    if score==0.0: # Check if it's pure and therefore an edge
        # return an edge
        vals = uniquecounts(df,col = val_col).iloc[0,:]
#         print(vals[val_col],vals['Counts'])
        return TreeNode(results=(vals[val_col],vals['Counts']),depth=depth) 
    
    # Find the best split
    set1, set2, best_column, best_criteria = findbest(df,decision_function,val_col)
    
#     display(set1.head())
#     display(set2.head())
    true_branch = buildTree(set1,decision_function,val_col,depth+1)
    false_branch = buildTree(set2,decision_function,val_col,depth+1)
    # Build this into a new tree
    return TreeNode(depth,column=best_column,value=best_criteria,true_branch=true_branch,false_branch=false_branch)

    
#     # Check if there was no split to find (i.e. Pure set)
#     if type(set1)==pd.core.frame.DataFrame:    
#         display(set1.head())
#         display(set2.head())
#         true_branch = buildTree(set1,decision_function,val_col,depth+1)
#         false_branch = buildTree(set2,decision_function,val_col,depth+1)
#         # Build this into a new tree
#         return TreeNode(depth,column=best_column,value=best_criteria,true_branch=true_branch,false_branch=false_branch)

#     else:
#         # return an edge
#         vals = uniquecounts(df,col = val_col).iloc[0,:]
# #         print(vals[val_col],vals['Counts'])
#         return TreeNode(results=(vals[val_col],vals['Counts']),depth=depth)  
    
    
tree = buildTree(df_format)
# print(tree)

Initial Score: 0.3648

Best selection was:
For Column Shoe Slipperiness, best gain was 0.0901 for split at 1.5
First Group (15 rows) Score: 0.1244 and Second Group (10 rows) Score: 0.5
Initial Score: 0.12444444444444444

Best selection was:
For Column Shoe Slipperiness, best gain was 0.0178 for split at 2.5
First Group (10 rows) Score: 0.0 and Second Group (5 rows) Score: 0.32
Initial Score: 0.32000000000000006

Best selection was:
For Column Floor Slipperiness, best gain was 0.0533 for split at 1.5
First Group (3 rows) Score: 0.4444 and Second Group (2 rows) Score: 0.0
Initial Score: 0.4444444444444444

Best selection was:
For Column Floor Slipperiness, best gain was 0.4444 for split at 2.5
First Group (2 rows) Score: 0.0 and Second Group (1 rows) Score: 0.0
Initial Score: 0.5

Best selection was:
For Column Floor Slipperiness, best gain was 0.3333 for split at 1.5
First Group (6 rows) Score: 0.2778 and Second Group (4 rows) Score: 0.0
Initial Score: 0.2777777777777778

Best selection

# Visualise The Tree

What questions are being asked of the data?

Calling the print function on the tree will trigger the "__str__" function causing it to output its contents

In [146]:
# How deep is the tree? (i.e how many questions deep does it go?)
tree.return_depth()

4

In [147]:
print(tree)

1. Shoe Slipperiness>=1.5?:
    2. Shoe Slipperiness>=2.5?:
        Results -> Fall : 10 samples
    2. Shoe Slipperiness<2.5?:
        3. Floor Slipperiness>=1.5?:
            4. Floor Slipperiness>=2.5?:
                Results -> Fall : 2 samples
            4. Floor Slipperiness<2.5?:
                Results -> No Fall : 1 samples
        3. Floor Slipperiness<1.5?:
            Results -> Fall : 2 samples
1. Shoe Slipperiness<1.5?:
    2. Floor Slipperiness>=1.5?:
        3. Floor Slipperiness>=2.5?:
            Results -> Fall : 4 samples
        3. Floor Slipperiness<2.5?:
            4. Shoe Slipperiness>=0.5?:
                Results -> No Fall : 1 samples
            4. Shoe Slipperiness<0.5?:
                Results -> Fall : 1 samples
    2. Floor Slipperiness<1.5?:
        Results -> No Fall : 4 samples


# Make a prediction

This will take a sliced row of the dataframe (a Series) and will apply the relevant questions to it

**Note** It is expecting numerical values to query and not categories. This code can be improved to enable this.

In [139]:
# Make a prediction
# For this we want an array
display(df_format.head(10))

test = df_format.loc[2,:] # Let us take the third row (labelled as 2)
display(test) # Display what the row looks likes

# Make a prediction and return it
tree.predict(test)

,Shoe Slipperiness,Floor Slipperiness,value
0,0,0,No Fall
1,1,0,No Fall
2,2,0,Fall
3,3,0,Fall
4,4,0,Fall
5,0,1,No Fall
6,1,1,No Fall
7,2,1,Fall
8,3,1,Fall
9,4,1,Fall


Shoe Slipperiness        2
Floor Slipperiness       0
value                 Fall
Name: 2, dtype: object

'Fall'

In [26]:
val=None
val==None

True

In [59]:
class Test:
    def __init__(self,depth,values="Hello"):
        self.depth = depth
        self.values=values
        
    def __str__(self):
        if self.values=='Hello':
            return str(self.values)
        else:
            return str(self.depth*"  "+self.values)
    
    def predict(self):
        pass
        
all_val = Test(6,"Hi")
print(all_val)

            Hi


In [17]:
all_val

In [18]:
print(all_val)

      Hello
